In [26]:
# Librerias necesarias
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [27]:
# Rutas de los datos:
ruta_games='Datasets originales/steam_games.json.gz'
ruta_user_reviews='Datasets originales/user_reviews.json.gz'
ruta_user_items='Datasets originales/users_items.json.gz'

## Games:
games_orig = []
with gzip.open(ruta_games, 'rt', encoding='utf-8') as archivo:
    lineas = archivo.readlines()
    for linea in lineas:
        games_orig.append(linea) 

# Quitamos el '\n' al final de cada linea:
for i in range(0,len(games_orig)):
    games_orig[i]= games_orig[i].replace("\n","")
# Transformo cada una de las cadenas a diccionario de Python:
for i in range(0,len(games_orig)):
    games_orig[i]= json.loads(games_orig[i])
    
# Transformamos a DataFrame:
games_orig = pd.DataFrame(games_orig)
games_orig.shape

(120445, 13)

In [28]:
## Reviews:
user_reviews = []
with gzip.open(ruta_user_reviews, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    user_reviews.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_reviews_orig = pd.DataFrame(user_reviews)
user_reviews_orig.shape

(25799, 3)

In [29]:
## Items:
user_items = []
with gzip.open(ruta_user_items, 'rt', encoding="utf-8") as archivo:
    lineas = archivo.readlines()
for linea in lineas:
    user_items.append(ast.literal_eval(linea))
    
# Transformamos a DataFrame:
user_items_orig = pd.DataFrame(user_items)
user_items_orig.shape

(88310, 5)

#### 1. Extracción

In [123]:
games = games_orig.copy()
user_reviews = user_reviews_orig.copy()
user_items = user_items_orig.copy()

#### 2. Transformación

##### 2.1 Para la consulta n° 1: PlayTimeGenre

In [124]:
games.tail(2) # de este df me sirve la columna genre y id

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


In [125]:
user_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [126]:
# Extraigo solo las columnas que voy a usar

reviews = user_reviews[['user_id','reviews']]
reviews.head()

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,"[{'funny': '3 people found this review funny',..."


In [128]:
pd.json_normalize(reviews['reviews'][0])

,funny,posted,last_edited,item_id,helpful,recommend,review
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


In [129]:
reviews_desanidado= pd.DataFrame(columns=['user_id','funny','posted','last_edited','item_id','helpful','recommend','review'])

for i, registro in enumerate(reviews['reviews']): # para c/elemento de c/registro de la columna reviews['reviews']
    registro_desanidado = pd.json_normalize(registro)
    registro_desanidado['user_id'] = reviews['user_id'][i]
    reviews_desanidado = pd.concat([reviews_desanidado,registro_desanidado],ignore_index=True)


In [132]:
print(reviews_desanidado.shape)
reviews_desanidado.head()

(59305, 8)


,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [133]:
user_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [135]:
# Extraigo solo las columnas que voy a usar:
items = user_items[['user_id','items_count','items']]
items.head()

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [144]:
items_desanidado= pd.DataFrame(columns=['user_id','item_id','item_name','playtime_forever','playtime_2weeks'])

for i, registro in enumerate(items['items']): # para c/elemento de c/registro de la columna reviews['reviews']
    registro_desanidado = pd.json_normalize(registro) # creo un pequeño dataframe que contendra todos los json que estaban en la lista de ese registro
    registro_desanidado['user_id'] = items['user_id'][i] # para ese registro desanidado creo una columna user_id que tendra el valor que tiene el dataframe madre en la columna user_id en la posicion del registro i
    items_desanidado = pd.concat([items_desanidado,registro_desanidado],ignore_index=True)